In [ ]:
# This is only a very early prototype of the interactive widget using three.js

import bpy
from pathlib import Path
import molecularnodes as mn
from ipymolecularnodes import BlenderInteractiveWidget

# Ensure the glTF exporter is activated
bpy.ops.preferences.addon_enable(module="io_scene_gltf2")

# Delete all mesh objects from the scene
bpy.ops.object.select_all(action="DESELECT")
bpy.ops.object.select_by_type(type="MESH")
bpy.ops.object.delete()

obj = mn.load.molecule_rcsb("6N2Y", starting_style="ball_and_stick", center_molecule=True)
obj.scale = [10, 10, 10]

"""
Applies the modifiers on the modifier stack

This will realise the computations inside of any Geometry Nodes modifiers, ensuring
that the result of the node trees can be compared by looking at the resulting
vertices of the object.
"""
bpy.context.view_layer.objects.active = obj
for modifier in obj.modifiers:
    bpy.ops.object.modifier_apply(modifier=modifier.name)

# Exporting the model to .glb format for Three.js
# Specify the path where you want to save the exported model
gltf_path = Path() / "_MY_MODEL_DATA.gltf"

# Select only the object
bpy.ops.object.select_all(action="DESELECT")
obj.select_set(True)

# Export the selected obj object to a .glb file
bpy.ops.export_scene.gltf(
    filepath=str(gltf_path), export_format="GLTF_EMBEDDED", use_selection=True
)

data = gltf_path.read_text()

import json
json_data = json.loads(data)

# gltf_path.unlink()

In [ ]:
b = BlenderInteractiveWidget(gltf_data = data)
b

In [ ]:
# Modify the JSON object
new_material = {
    "doubleSided": True,
    "name": "OrangeMaterial",
    "pbrMetallicRoughness": {
        "baseColorFactor": [1, 0.5, 0, 1],
        "metallicFactor": 0,
        "roughnessFactor": 0.5
    }
}

json_data["materials"][0] = new_material

# Convert the JSON object back to a string
data_yellow = json.dumps(json_data, indent=4)

In [ ]:
b3 = BlenderInteractiveWidget(gltf_data = data_yellow)
b3

In [ ]:
type(data_yellow)